In [23]:
import database
import importlib
import os
from datetime import datetime

from snowballing.operations import load_work, load_citations, reload
from snowballing.operations import work_by_varname, load_work_map_all_years
from snowballing.models import Place, DB
from snowballing.jupyter_utils import idisplay, work_button, new_button
from snowballing.selenium_scholar import SeleniumScholarQuerier
from snowballing.dbmanager import set_attribute

def pages(work):
    pp = getattr(work, 'pp', '')
    if not pp:
        return 0
    split = list(map(int, pp.split('--')))
    if len(split) == 1:
        return split[0]
    return split[-1] - split[0] + 1

## Check Place

In [2]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if hasattr(w, "place1")
    or not isinstance(w.place, Place)
]
len(result)

0

In [3]:
[p for p in DB.places() if not hasattr(p, "type")]

[]

In [4]:
[(p, p.type) for p in DB.places()
 if p.type not in {
     "Conference", 
     "Journal", 
     "Magazine", 
     "Sponsor",
     "Proceedings",
     "Archive",
     "Tech Report",
     "Unpublished",
     "Thesis",
     "Patent",
     "Lang",
     "Web",
     "Standard",
     "Book",
     "Other",
 }]

[(PRBook, 'Peer Reviewed Book'), (PRBook, 'Peer Reviewed Book')]

## Check IPAW papers

In [5]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if (w.category == "fullpaper" and w.place.acronym != "IPAW")
    or (w.category == "regular" and w.place.acronym != "IPAW")
    or (w.category == "short" and w.place.acronym != "IPAW")
    or (w.category == "demo" and w.place.acronym != "IPAW")
    or (w.category == "poster" and w.place.acronym != "IPAW")
    or (w.category == "keynote" and w.place.acronym != "IPAW")
    or (w.category == "latebreaking" and w.place.acronym != "IPAW")
    or (w.place.acronym == "IPAW" and w.category not in {
        "fullpaper", "regular", "short", "demo", "poster", "keynote", "latebreaking"
    })
]
len(result)

0

## Check pages

In [6]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if not hasattr(w, "pp")
    if w.category in {
        "fullpaper", "regular", "short", "demo", "poster", "latebreaking"
    }
    
]
len(result)

0

## Check Name

In [ ]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if not hasattr(w, "name")
    or w.name.endswith(".")
]
len(result)

## Check Authors

In [ ]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if not hasattr(w, "authors")
    or w.authors.endswith(".")
    and not w.authors.endswith(" al.")
    and not w.authors[-3] in "- " and w.authors[-2].isalpha()
]
len(result)

## Scholar

In [ ]:
querier = None
reload()
worklist = sorted(
    [k for k, w in load_work_map_all_years() if not getattr(w, "scholar_ok", False)],
    key=lambda x: (int(x[-5:-1]), x)
)
len(worklist)

In [ ]:
if worklist and querier is None:
    querier = SeleniumScholarQuerier()
    querier.apply_settings(10, 4)

In [ ]:
from snowballing.snowballing import ScholarUpdate
supdate = ScholarUpdate(querier, worklist, force=False)

In [ ]:
supdate.browser()

In [ ]:
# Temp
set_attribute('wu2016a', 'cluster_id', '14505887097699069314')
set_attribute('wu2016a', 'scholar', 'http://scholar.google.com/scholar?cites=14505887097699069314&as_sdt=2005&sciodt=0,5&hl=en')
set_attribute('wu2016a', 'scholar_ok', True)
None

In [ ]:
{w.scholar_ok for k, w in load_work_map_all_years() if getattr(w, "scholar_ok", False)}

## Forward

Finding scholar cluster: http://webapps.stackexchange.com/questions/45333/how-to-create-a-citation-alert-for-a-paper-without-citation-in-google-scholar

In [41]:
def set_snowball_date_button(key, date):
    def click(w):
        set_attribute(key, "snowball", "datetime({0.year}, {0.month}, {0.day})".format(date))
    return new_button("Set date", click)
    
date = datetime(2018, 5, 17)
reload()
result = [
    idisplay(
        work_button(w),
        set_snowball_date_button(k, date),
        '"{}", citation_file="{}"'.format(k, getattr(w, 'citation_file', w.metakey)) if hasattr(w, 'scholar') else w.name,
        label=False
    )
    for k, w in load_work_map_all_years()
    if w.place.acronym == "IPAW"
    if pages(w) >= 6
    #if w.category in {"fullpaper", "regular"}
    if not hasattr(w, 'snowball') or w.snowball != date
    if w.year >= 2008
]
len(result)

0

## Dash

In [8]:
def check_dash(ele):
    k, w = ele
    def create_button(attr):
        def click(_):
            set_attribute(k, attr, str(getattr(w, attr)).replace("-", "--"))
        return new_button("Set " + attr, click)
    reasons = []
    if w.place.name in ("Thesis", "Tech Report"):
        return k, w, reasons
    if isinstance(getattr(w, 'pp', ''), int):
        reasons.append(create_button('pp'))
    elif getattr(w, 'pp', '').count('-') not in (0, 2):
        reasons.append(create_button('pp'))
    if isinstance(getattr(w, 'volume', ''), int):
        reasons.append(create_button('volume'))
    elif getattr(w, 'volume', '').count('-') not in (0, 2):
        reasons.append(create_button('volume'))
    if isinstance(getattr(w, 'number', ''), int):
        reasons.append(create_button('number'))
    elif getattr(w, 'number', '').count('-') not in (0, 2):
        reasons.append(create_button('number'))
    return k, w, reasons

reload()
result = [
    idisplay(work_button(w), *r)
    for k, w, r in map(check_dash, load_work_map_all_years())
    if r
]
len(result)

0